In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_save_path: Path
    token_embed_path: Path
    epochs:int
    classes:int
    learning_rate:float
    input_shape:tuple
    input_dtype: int
    params: any
    random_state:int


In [6]:
from typing import Union
from pathlib import Path
from sentimentanalyzer.utils.common import read_yaml, create_directories, set_seed# adjust import as needed
from sentimentanalyzer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
from dataclasses import dataclass

In [7]:

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Union[str, Path] = CONFIG_FILE_PATH,
        params_filepath: Union[str, Path] = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        print(">>> CONFIG CONTENTS:", self.config)
        print(">>> CONFIG KEYS:", list(self.config.keys()))

        self.params = read_yaml(params_filepath)
        print(">>> PARAMS CONTENTS:", self.params)
        print(">>> PARAMS KEYS:", list(self.params.keys()))

        # Create root directory if exists
        if 'artifacts_root' in self.config:
            create_directories([self.config.artifacts_root])
        else:
            raise KeyError("Missing 'artifacts_root' in config.yaml")


        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """
        Read the `model_trainer` section of the config and
        combine it with training params into a ModelTrainerConfig.
        """
        config = self.config.model_trainer

        # make sure the model‐trainer folder exists
        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path = config.data_path,
            model_save_path=config.model_save_path,
            token_embed_path=config.token_embed_path,
            epochs=self.params.epochs,
            classes=self.params.classes,
            learning_rate=self.params.learning_rate,
            input_shape=self.params.input_shape,
            input_dtype=self.params.input_dtype,
            params=self.params,
            random_state= self.params.random_state
        )


In [8]:
import ast
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers


class ModelTrainer:
    def __init__(self, config: 'ModelTrainerConfig'):
        self.config = config
        self.params = config.params

        set_seed(self.config.random_state)
        # (You already loaded these embeddings earlier if needed)
        self.token_embed = np.load(self.config.token_embed_path)

    def train(self):
        data_path = Path(self.config.data_path)
        
        # 1) Ensure input_shape is a tuple
        input_shape = self.config.input_shape
        if isinstance(input_shape, str):
            input_shape = tuple(ast.literal_eval(input_shape))


        # 2) Define Input layer for precomputed float32 embeddings
        inputs = layers.Input(
            shape=input_shape,    # e.g. (163, 107)
            dtype=tf.float32,     # embeddings are float32
            name="embedding_input"
        )

        # 3) Conv + pooling
        x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(inputs)
        x = layers.GlobalAveragePooling1D()(x)

        # 4) Classification head — use `classes` from params
        outputs = layers.Dense(
            units=self.params.classes,
            activation="softmax",
            name="classifier"
        )(x)

        # 5) Build & compile
        model = models.Model(inputs=inputs, outputs=outputs, name="EmbeddingConv1DModel")
        model.compile(
            optimizer=optimizers.Adam(learning_rate=self.params.learning_rate),
            loss=losses.SparseCategoricalCrossentropy(), 
            metrics=["accuracy"]
        )

    
        
        model.summary()

        # 6) Load tf.data datasets (must yield (embeddings, labels))
        #root = Path(self.config.root_dir)
        
        train_ds = tf.data.experimental.load(str(data_path / "train_ds"))
        valid_ds = tf.data.experimental.load(str(data_path / "valid_ds"))


        # 7) Train
        model.fit(
            train_ds,
            epochs=self.params.epochs,
            validation_data=valid_ds
        )

        # 8) Save
        model.save(self.config.model_save_path)

        return model


In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer = model_trainer.train()
    
except Exception as e:
    raise e

[2025-06-14 22:39:52,599: INFO: common: yaml file: config\config.yaml loaded successfully]
>>> CONFIG CONTENTS: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/Spencer0013/NLP-Text-Summarizer-Project/raw/refs/heads/main/Dataa.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_preprocessing': {'root_dir': 'artifacts/data_preprocessing', 'ingestion_dir': 'artifacts/data_ingestion', 'output_dir': 'artifacts/data_preprocessing'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'transformer_data': 'artifacts/data_transformation/transformer_data', 'data_path_train': 'artifacts/data_preprocessing/train_clean.csv', 'model_name': 'bert-base-uncased', 'data_path_test': 'artifacts/data_preprocessing/test_clean.csv', 'transformed_token_embedding_path': 'artifacts/data_transformation/token_embeddings.npy'}, 'model_trainer': {'root_dir': 'artifa

c:\Users\ainao\anaconda3\envs\sentA\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
